### Add upstream PFAFIDs to merged shapefile

* Purpose of script: Create a csv file with all PFAFID's updstream
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170822

Revisited to apply normalization to database structure. 

In [6]:
%matplotlib inline
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

('Y2017M11D17', 'UTC 17:07')


'2.7.13 |Continuum Analytics, Inc.| (default, Dec 20 2016, 23:09:15) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [7]:
SCRIPT_NAME = "Y2017M08D22_RH_Upstream_V01"

S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output/"
EC2_INPUT_PATH = "/volumes/data/Y2017M08D22_RH_Upstream_V01/input/"
EC2_OUTPUT_PATH = "/volumes/data/Y2017M08D22_RH_Upstream_V01/output/"
INPUT_FILENAME = "hybas_lev06_v1c_merged_fiona_V01.shp"
OUTPUT_FILENAME = "hybas_lev06_v1c_merged_fiona_V01.csv"
OUTPUT_FILENAME_AUGMENTED = "hybas_lev06_v1c_merged_fiona_upstream_V01.csv"
OUTPUT_FILENAME_AUGMENTED2 = "hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv"

In [4]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

Copying the files from S3 to EC2 since Shapefiles consist of multiple files

In [5]:
!aws s3 cp {S3_INPUT_PATH} {EC2_INPUT_PATH} --recursive --exclude *.tif

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev00_v1c_merged_fiona_V01.cpg to ../../../../data/Y2017M08D22_RH_Upstream_V01/input/hybas_lev00_v1c_merged_fiona_V01.cpg
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.cpg to ../../../../data/Y2017M08D22_RH_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_V01.cpg
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev00_v1c_merged_fiona_V01.prj to ../../../../data/Y2017M08D22_RH_Upstream_V01/input/hybas_lev00_v1c_merged_fiona_V01.prj
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.prj to ../../../../data/Y2017M08D22_RH_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_V01.prj
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v

In [8]:
ec2InputFile = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME)
ec2OutputFile = os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME)

In [9]:
!ogr2ogr -f CSV {ec2OutputFile} {ec2InputFile}

Test if the CSV is correct (without geometry)

In [10]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

upload: ../../../../data/Y2017M08D22_RH_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_V01.csv to s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_V01.csv


## Functions

In [11]:
def find_upstream_catchments(basin_ids, df):
    all_up_catchments =[]
    for i in np.arange(1, df.shape[0], 1):
        if not basin_ids:
            break
        else:
            up_catchments_adjacent = []
            for bid in basin_ids:
                bid = np.int64(bid)
                up_ids_idx = df[df['NEXT_DOWN'] == bid]
                up_ids_idx = up_ids_idx.index.tolist()
                for idx in up_ids_idx :
                    up_id = np.int64(df.HYBAS_ID[idx])
                    up_catchments_adjacent.append(up_id)
            basin_ids = up_catchments_adjacent
            all_up_catchments = all_up_catchments + basin_ids

    all_up_catchments_PFAF = df.loc[df.HYBAS_ID.isin(all_up_catchments)].PFAF_ID.tolist()
    #all_up_catchments_PFAF = map(lambda(x): np.int64(x), all_up_catchments_PFAF)    
    return all_up_catchments,all_up_catchments_PFAF

def generate_dictionary(df, outputLocation):
    df_temp = df
    up_catchments = []
    up_catchments_PFAF = []
    for bid in df.HYBAS_ID:
        bid = np.int64(bid)
        (all_up_catchments, all_up_catchments_PFAF) = find_upstream_catchments([bid], df)
        up_catchments.append(all_up_catchments)
        up_catchments_PFAF.append(all_up_catchments_PFAF)
        print(bid)


    df_temp['Upstream_HYBAS_IDs'] = up_catchments
    df_temp['Upstream_PFAF_IDs'] = up_catchments_PFAF
    df_temp.to_csv(outputLocation)
    print("DONE!")

In [12]:
df = pd.read_csv(ec2OutputFile)

In [13]:
df.head()

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT
0,6060000010,0,6060000010,6060000010,0.0,0.0,4317.4,4317.4,611001,0,1,0,1
1,6060000200,0,6060000200,6060000200,0.0,0.0,35995.5,35996.7,611002,0,0,1,2
2,6060000210,0,6060000210,6060000210,0.0,0.0,443.9,443.9,611003,0,1,0,3
3,6060000240,0,6060000240,6060000240,0.0,0.0,2186.3,2186.3,611004,0,0,1,4
4,6060000250,0,6060000250,6060000250,0.0,0.0,6533.8,6533.8,611005,0,1,0,5


In [11]:
outputLocationAugentedCSV =  os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME_AUGMENTED)
outputLocationAugentedCSV2 = os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME_AUGMENTED2)

In [12]:
generate_dictionary(df, outputLocationAugentedCSV)

6060000010
6060000200
6060000210
6060000240
6060000250
6060000450
6060000460
6060000580
6060000590
6060000740
6060068100
6060068110
6060070260
6060070270
6060070390
6060070330
6060069460
6060069430
6060078120
6060078150
6060089660
6060089740
6060094950
6060094960
6060102120
6060102080
6060107990
6060107870
6060122060
6060122070
6060130930
6060131030
6060136120
6060136040
6060000750
6060000780
6060000790
6060001110
6060001120
6060001590
6060001600
6060001910
6060001920
6060001940
6060001950
6060001970
6060001980
6060002270
6060002280
6060002860
6060002870
6060003100
6060003110
6060003210
6060003220
6060003680
6060003690
6060003730
6060003740
6060003800
6060003810
6060004000
6060004010
6060004030
6060004040
6060004060
6060004070
6060004470
6060075390
6060075410
6060079390
6060079310
6060093530
6060093540
6060101210
6060101170
6060084190
6060084230
6060089790
6060089780
6060084040
6060084130
6060084460
6060084380
6060084750
6060084790
6060097370
6060097380
6060086300
6060086370
6060123500

6060458930
6060459050
6060451380
6060451390
6060476790
6060476800
6060462190
6060462200
6060480090
6060480100
6060481840
6060481630
6060486940
6060486950
6060490170
6060490160
6060477950
6060478100
6060498620
6060498490
6060496260
6060496350
6060499330
6060499430
6060486490
6060486650
6060517980
6060517810
6060509200
6060509190
6060527670
6060527680
6060516540
6060516530
6060497960
6060498060
6060522650
6060522510
6060528780
6060528770
6060543870
6060543860
6060550960
6060551090
6060546240
6060546360
6060550000
6060549710
6060565980
6060565970
6060563680
6060563830
6060583880
6060583870
6060571240
6060571360
6060573000
6060573010
6060579600
6060579610
6060590040
6060590110
6060603760
6060603770
6060572890
6060572820
6060596300
6060596160
6060594780
6060594790
6060586940
6060587040
6060615450
6060599670
6060615600
6060599570
6060619500
6060619510
6060594270
6060594380
6060622980
6060622990
6060624770
6060624870
6060007960
6060008010
6060008020
6060008110
6060008120
6060008140
6060008150

6060677380
6060661150
6060661140
6060644240
6060644230
6060675170
6060675380
6060673380
6060673370
6060654680
6060654560
6060742400
6060742440
6060644880
6060644890
6060719760
6060719850
6060748650
6060748810
6060597460
6060597540
6060601680
6060601730
6060016980
6060017370
6060017380
6060911700
6060911680
6060907520
6060907490
6060906970
6060906910
6060916780
6060916830
6060017390
6060017530
6060017540
6060017970
6060017980
6060018280
6060018290
6060018410
6060018420
6060018440
6060018450
6060018500
6060018510
6060018560
6060018570
6060018850
6060933820
6060933800
6060925990
6060925960
6060923460
6060923410
6060929600
6060929480
6060919110
6060919090
6060927460
6060927470
6060917060
6060917010
6060913910
6060913900
6060903990
6060903930
6060901010
6060901000
6060898890
6060898920
6060918580
6060918550
6060898190
6060898150
6060903070
6060903110
6060902800
6060902690
6060891020
6060890970
6060909950
6060910020
6060885150
6060885110
6060877660
6060877590
6060874710
6060874640
6060874330

4060370730
4060370560
4060362240
4060362250
4060348030
4060348020
4060327320
4060327110
4060320440
4060320450
4060311990
4060311910
4060332450
4060332620
4060332910
4060333000
4060314260
4060314110
4060327490
4060327650
4060006580
4060006620
4060006630
4060006670
4060006680
4060006750
4060006760
4060006840
4060006850
4060006940
4060007080
4060460860
4060460870
4060421080
4060420910
4060419170
4060419030
4060398360
4060398160
4060007090
4060007380
4060007400
4060007410
4060483160
4060483080
4060479550
4060479560
4060486110
4060486260
4060486970
4060486980
4060487710
4060487880
4060489140
4060488910
4060487620
4060487460
4060491290
4060491440
4060493260
4060493450
4060511760
4060511540
4060497590
4060497290
4060443420
4060443160
4060449240
4060449230
4060509830
4060509930
4060567380
4060567390
4060461150
4060461010
4060007420
4060007660
4060007670
4060007850
4060577130
4060577480
4060579160
4060579370
4060615410
4060615400
4060620090
4060620160
4060627250
4060627620
4060625630
4060625500

4060943790
4060943690
4060991510
4060991520
4060950490
4060950570
4060947780
4060947770
4060976580
4060976590
4060930100
4060929890
4060927760
4060927750
4060913570
4060913560
4060903190
4060902890
4060876090
4060876200
4060023820
4060024190
4060024780
4060024790
4060024890
4060024900
4060025260
4060025270
4060025330
4060025340
4060025450
4060975160
4060975150
4060967140
4060967130
4060960200
4060960480
4060956230
4060956220
4060949680
4060949690
4060940160
4060940040
4060900670
4060900680
4060929850
4060929710
4060921230
4060921220
4060929740
4060929440
4060912930
4060912840
4060886350
4060886200
4060911800
4060911950
4060883850
4060884090
4060866220
4060866230
4060947390
4060947550
4060901970
4060901870
4060909900
4060909770
4060901010
4060900840
4060884290
4060884570
4060883390
4060883720
4060859640
4060859650
4060914430
4060914420
4060876180
4060876190
4060872360
4060872230
4060826770
4060826780
4060905060
4060905050
4060863950
4060863940
4060903120
4060902870
4060870060
4060870190

...

7060585700
7060585550
7060420340
7060420200
7060512440
7060512450
7060415850
7060415980
7060597680
7060597760
7060407800
7060407660
7060406580
7060406640
7060505760
7060505540
7060478560
7060478320
7060435860
7060435970
7060523200
7060523060
7060432240
7060432540
7060537550
7060537830
7060529340
7060529460
7060468930
7060468870
7060376630
7060376640
7060470820
7060470900
7060535230
7060534960
7060414650
7060414530
7060412280
7060411940
7060513230
7060513060
7060448300
7060448140
7060349590
7060349950
7060419360
7060419260
7060485210
7060485360
7060424530
7060424300
7060404530
7060404520
7060397370
7060397380
7060429220
7060429360
7060372880
7060372750
7060454960
7060454870
7060456180
7060456330
7060342130
7060341960
7060332670
7060332680
7060328250
7060328380
7060364590
7060364600
7060330560
7060330450
7060373260
7060373250
7060346150
7060346030
7060380010
7060380120
7060416830
7060416820
7060333810
7060333560
7060047070
7060047210
7060047220
7060047480
7060047490
7060047770
7060047780

8060110500
8060111190
8060111120
8060007190
8060007630
8060007640
8060007690
8060007700
8060007830
8060007840
8060008020
8060008030
8060008230
8060008240
8060008380
8060089970
8060089980
8060090390
8060090400
8060096520
8060096530
8060097430
8060097530
8060008390
8060008470
8060008480
8060008510
8060008520
8060008550
8060008560
8060008630
8060008640
8060008900
8060008970
8060008980
8060009130
8060009140
8060009240
8060009250
8060009460
8060009470
8060009560
8060112340
8060112350
8060122010
8060122100
8060155680
8060155780
8060157910
8060158000
8060160210
8060160330
8060159800
8060159890
8060148430
8060148510
8060164230
8060164290
8060183170
8060183070
8060194590
8060194580
8060196780
8060196880
8060170630
8060170460
8060155650
8060155550
8060155560
8060155470
8060231560
8060231700
8060148890
8060148880
8060147790
8060147670
8060243400
8060243490
8060150960
8060150880
8060147910
8060147850
8060162560
8060162570
8060252610
8060252760
8060254850
8060254970
8060152450
8060152640
8060176280

9060006570
9060006580
9060007250
9060007260
9060007310
9060007320
9060007510
9060007520
9060007570
9060007580
9060007930
9060007940
9060007950
9060008450
9060008460
9060008490
9060008500
9060008590
9060008600
9060009250
9060009260
9060009300
9060009310
9060009880
9060120760
9060120830
9060120590
9060120560
9060118480
9060118520
9060118610
9060118590
9060009890
9060010370
9060010410
9060109660
9060109730
9060111590
9060111620
9060103730
9060103700
9060103260
9060103220
9060010430
9060010550
9060100080
9060100020
9060100090
9060100030
9060097020
9060096990
9060094910
9060094850
9060094780
9060094730
9060098410
9060098480
9060092650
9060092570
9060092400
9060092340
9060101230
9060101270
9060101310
9060101360
9060091390
9060091450
9060089660
9060089740
9060010560
9060010670
9060010680
9060011090
9060011100
9060011220
9060082000
9060081990
9060079140
9060079120
9060078520
9060078540
9060077450
9060077430
9060011230
9060011510
9060011530
9060011740
9060068800
9060068750
9060067310
9060067440

3060664370
3060664490
3060665280
3060665340
3060666800
3060666660
3060671160
3060671260
3060676020
3060675920
3060679750
3060679760
3060686770
3060686590
3060692570
3060692580
3060685820
3060685780
3060680250
3060680430
3060548610
3060548460
3060704050
3060704040
3060696590
3060696460
3060673930
3060674030
3060690740
3060690650
3060714210
3060714200
3060697190
3060697120
3060716290
3060716280
3060684450
3060684440
3060681050
3060681160
3060710300
3060710230
3060718770
3060718850
3060679010
3060678890
3060698660
3060698770
3060700380
3060700640
3060725910
3060725930
3060719570
3060719560
3060719950
3060719960
3060721920
3060722010
3060719880
3060719840
3060668420
3060668430
3060004750
3060005240
3060005440
3060049480
3060049490
3060051810
3060051820
3060069600
3060069590
3060078870
3060078880
3060088060
3060088050
3060076220
3060076210
3060091010
3060091000
3060074650
3060074660
3060073760
3060073910
3060069430
3060069350
3060112100
3060112250
3060115500
3060115600
3060005450
3060005480

3060020980
3060021030
3060021580
3060021630
3060021920
3060021930
3060022340
3060022360
3060022480
3060022640
3060022680
3060023360
3060023370
3060023380
3060023650
3060023660
3060023670
3060023700
3060023800
3060024010
3060024170
3060024180
3060024310
3060704250
3060704320
3060713270
3060713340
3060713450
3060713510
3060697710
3060697620
3060024320
3060024330
3060679080
3060679070
3060679200
3060678950
3060678960
3060678860
3060679090
3060024340
3060024350
3060024360
3060024370
3060024380
3060024390
3060024400
3060024410
3060024420
3060024430
3060024440
3060024450
3060024460
3060024470
3060024480
3060024490
3060024510
3060024530
3060024540
3060024570
3060024580
3060024610
3060024620
3060024640
3060024670
3060024690
3060024710
3060024780
3060024830
3060024860
3060024870
3060024890
3060024960
3060024970
3060025010
3060025070
3060025110
3060025150
3060025170
3060025180
3060025240
3060025320
3060025460
3060025570
3060026330
3060026650
3060027750
DONE!


In [13]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

upload: ../../../../data/Y2017M08D22_RH_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_V01.csv to s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_V01.csv
upload: ../../../../data/Y2017M08D22_RH_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_V01.csv to s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_V01.csv


In [14]:
df.head()

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,Upstream_HYBAS_IDs,Upstream_PFAF_IDs
0,6060000010,0,6060000010,6060000010,0.0,0.0,4317.4,4317.4,611001,0,1,0,1,[],[]
1,6060000200,0,6060000200,6060000200,0.0,0.0,35995.5,35996.7,611002,0,0,1,2,[],[]
2,6060000210,0,6060000210,6060000210,0.0,0.0,443.9,443.9,611003,0,1,0,3,[],[]
3,6060000240,0,6060000240,6060000240,0.0,0.0,2186.3,2186.3,611004,0,0,1,4,[],[]
4,6060000250,0,6060000250,6060000250,0.0,0.0,6533.8,6533.8,611005,0,1,0,5,[],[]


In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)